In [1]:
import spotipy
import pandas as pd
import numpy as np
from spotipy.oauth2 import SpotifyOAuth
import time

In [2]:
client_id='c4ff820f994745288df451ea7d6940d5'
client_secret='be2297719e5e4fb698f112c4374ee3b1'
redirect_uri='https://open.spotify.com/'
token_url = "https://accounts.spotify.com/api/token"

## User Auth
- access spotify api via OAuth workflow

In [3]:
scope = 'playlist-read-private'

sp_oauth = SpotifyOAuth(client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri,scope=scope)
token_info = sp_oauth.get_cached_token() 
if not token_info:
    auth_url = sp_oauth.get_authorize_url()
    print(auth_url)
    response = input('Paste the above link into your browser, then paste the redirect url here: ')

    code = sp_oauth.parse_response_code(response)
    token_info = sp_oauth.get_access_token(code)

    token = token_info['access_token']

sp = spotipy.Spotify(auth=token, retries=10, requests_timeout=10)

https://accounts.spotify.com/authorize?client_id=c4ff820f994745288df451ea7d6940d5&response_type=code&redirect_uri=https%3A%2F%2Fopen.spotify.com%2F&scope=playlist-read-private


/var/folders/dy/m7tb4mx52sv0lxfvtf95srbc0000gn/T/ipykernel_92516/4118829141.py:11: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = sp_oauth.get_access_token(code)


In [4]:
# scope = 'playlist-read-private'

# oauth = spotipy.util.prompt_for_user_token(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, scope=scope)
# sp = spotipy.Spotify(auth=oauth)

In [5]:
if sp:
    print(sp.me())

{'display_name': 'amaya lim', 'external_urls': {'spotify': 'https://open.spotify.com/user/nightrainlily'}, 'href': 'https://api.spotify.com/v1/users/nightrainlily', 'id': 'nightrainlily', 'images': [{'url': 'https://i.scdn.co/image/ab67757000003b82180bd1d0c567a1e26710aeb8', 'height': 64, 'width': 64}, {'url': 'https://i.scdn.co/image/ab6775700000ee85180bd1d0c567a1e26710aeb8', 'height': 300, 'width': 300}], 'type': 'user', 'uri': 'spotify:user:nightrainlily', 'followers': {'href': None, 'total': 369}}


## Getting Playlists
- get all playlists via current_user_playlist method. use offsets to avoid rate limiting
- filter playlists using regex for only project playlists (numbered)
- create pandas df for analysis

In [6]:
from time import sleep
from tqdm import tqdm, trange

In [7]:
def get_all_playlists():
    playlist_count = 1000
    results = []
    offset = 0
    pbar = tqdm(total=1000)
    while offset < playlist_count:
        results.append(sp.current_user_playlists(limit=50, offset=offset))
        time.sleep(10)
        pbar.update(50)
        offset += 50
    pbar.close()
    return results
results = get_all_playlists()

100%|██████████| 1000/1000 [03:30<00:00,  4.74it/s]


In [8]:
def parse(playlist_json, target):
    lst = []
    for i in np.arange(len(playlist_json['items'])):
        target_value = playlist_json['items'][i]['track'][target]
        lst.append(target_value)
    return lst

def artist_parse(playlist_json, target):
    lst = []
    p = parse(playlist_json, target)
    for i in np.arange(len(p)):
        target_value = p[i][0]['id']
        lst.append(target_value)
    return lst
    

In [9]:
import re

playlist_data = []

for result in results:
    # pbar = tqdm(total=1000)
    for playlist in result['items']:
        reg = "^([0-9])+\s([a-z]+(\s?)([a-z]?))"
        # pbar.update(1)
        if re.search(reg, playlist['name']) != None and playlist['public'] == True:
            track_info = sp.playlist_tracks(playlist['id'], fields='items(track(id, name, artists))')
            playlist_data.append({
                'name' : playlist['name'],
                'track_ids' : parse(track_info, 'id'),
                'track_names' : parse(track_info, 'name'),
                'track_artists' : artist_parse(track_info, 'artists'),
                'id' : playlist['id'],
                'url' : playlist['external_urls']['spotify'],
                'image' : playlist['images'][0]['url']
            })
    # pbar.close()

playlists = pd.DataFrame(playlist_data)
# playlists.head().style.format({'image': lambda x : f'<img src="{x}" width="100px"/>'})
# playlists.head()

In [10]:
def pretty_tracks(col, name):
        all_l = []
        for i in np.arange(len(playlists[col])):
                row = playlists[col].values[i]
                l = []
                for j in row:
                        for k in np.arange(len(row)):
                                track = item[k]['track']
                                id = track[name]
                                l.append(id)
                all_l.append(l)
        return all_l

# playlists['track_ids'] = pretty_tracks('track_ids', 'id')
# playlists['track_names'] = pretty_tracks('track_names', 'name')

# playlists.head().style.format({'image': lambda x : f'<img src="{x}" width="100px"/>'})
playlists.head()


,name,track_ids,track_names,track_artists,id,url,image
0,849 find out,"[6vca0aMELa1kc6mIs6w39c, 5jOCZRdWItMJKacbZolEJ...","[toothache, Death Grip, The fish song, Trailbl...","[0nugNBwdWaptgIAsEtx1It, 4Ky0ZDJDfY7UBbTVYPyVK...",5oKwX8pOYWofuTNMKx4okR,https://open.spotify.com/playlist/5oKwX8pOYWof...,https://mosaic.scdn.co/640/ab67616d00001e02bd8...
1,848 innocence,"[6MzofobZt2dm0Kf1hTThFz, 5HwHdYpk3QiOUhsfWKYkF...","[Diet Pepsi, Passionfruit, tears in the club (...","[4gvjmrtzydbMpyJaXUtwvP, 2RqrWplViWHSGLzlhmDcb...",5sMFgzDK1HTjG2bxIyi9jI,https://open.spotify.com/playlist/5sMFgzDK1HTj...,https://mosaic.scdn.co/640/ab67616d00001e024e5...
2,847 sick joke,"[4ztA5OPlOzUvbE0Cgh2gV9, 505HhtdgN8lynQcJQCcIi...","[something will give, Ever Seen, Birdwatcher, ...","[3WWvqxymB3Nypxej8XcQKC, 35l9BRT7MXmM8bv2WDQiy...",3oldJCV4dggIdDCB8aFlFc,https://open.spotify.com/playlist/3oldJCV4dggI...,https://mosaic.scdn.co/640/ab67616d00001e0254c...
3,846 save me,"[3vlAqZvq7IYKOoOKwHGAxd, 0LXxEKF7GjUbGYCeBOPxb...","[double dog, Weirdwordsworms, Infinite Love wi...","[7HKjjFcvOZyupxbOMdmSwE, 4TZieE5978SbTInJswaay...",3g39V9zZwlt9ZYn7JKTOCj,https://open.spotify.com/playlist/3g39V9zZwlt9...,https://image-cdn-ak.spotifycdn.com/image/ab67...
4,845 what did you expect,"[0rxQTgJzd0HCUk4QUxSkjK, 74OfKhcmck9KUHBPmCaOt...","[Even If It Hurts (feat. Blood Orange), Drifti...","[1xcMOgFUM1IYZE22YjCvsL, 5FxD8fkQZ6KcsSYupDVoS...",0KgwFInw8xWYmtDQWayf2p,https://open.spotify.com/playlist/0KgwFInw8xWY...,https://image-cdn-ak.spotifycdn.com/image/ab67...


In [11]:
artists = playlists.pop('track_artists')
artists

0      [0nugNBwdWaptgIAsEtx1It, 4Ky0ZDJDfY7UBbTVYPyVK...
1      [4gvjmrtzydbMpyJaXUtwvP, 2RqrWplViWHSGLzlhmDcb...
2      [3WWvqxymB3Nypxej8XcQKC, 35l9BRT7MXmM8bv2WDQiy...
3      [7HKjjFcvOZyupxbOMdmSwE, 4TZieE5978SbTInJswaay...
4      [1xcMOgFUM1IYZE22YjCvsL, 5FxD8fkQZ6KcsSYupDVoS...
                             ...                        
922    [6xdRb2GypJ7DqnWAI2mHGn, 1r1uxoy19fzMxunt3ONAk...
923    [3WrFJ7ztbogyGnTHbHJFl2, 5B7NeaqVrmXPyF05C9tnZ...
924    [5QdyldG4Fl4TPiOIeMNpBZ, 3h9TfIgwhovQELlP2jj4x...
925    [2gqMBdyddvN82dzZt4ZF14, 2rDxtYUzTAYJJE3Bl3Z5I...
926    [4vAQ4M7vgItwBtmBTgRu48, 6Gl4Q3ePw6HKMfIOix5Qp...
Name: track_artists, Length: 927, dtype: object

In [12]:
def get_url(p_name):
    info = sp.playlist(playlists[playlists['name'] == p_name]['id'].item())
    url = info['external_urls']['spotify']
    return url

## Getting Song Data
- manipulate playlist df to get songs
- drop na values (locally stored files)

In [13]:
songs = playlists
songs['zipp'] = list(zip(songs['track_ids'], songs['track_names']))

In [14]:
def merge(z):
    return list(tuple(zip(z[0], z[1])))

songs['zipp'] = songs['zipp'].apply(merge)
songs.head()

,name,track_ids,track_names,id,url,image,zipp
0,849 find out,"[6vca0aMELa1kc6mIs6w39c, 5jOCZRdWItMJKacbZolEJ...","[toothache, Death Grip, The fish song, Trailbl...",5oKwX8pOYWofuTNMKx4okR,https://open.spotify.com/playlist/5oKwX8pOYWof...,https://mosaic.scdn.co/640/ab67616d00001e02bd8...,"[(6vca0aMELa1kc6mIs6w39c, toothache), (5jOCZRd..."
1,848 innocence,"[6MzofobZt2dm0Kf1hTThFz, 5HwHdYpk3QiOUhsfWKYkF...","[Diet Pepsi, Passionfruit, tears in the club (...",5sMFgzDK1HTjG2bxIyi9jI,https://open.spotify.com/playlist/5sMFgzDK1HTj...,https://mosaic.scdn.co/640/ab67616d00001e024e5...,"[(6MzofobZt2dm0Kf1hTThFz, Diet Pepsi), (5HwHdY..."
2,847 sick joke,"[4ztA5OPlOzUvbE0Cgh2gV9, 505HhtdgN8lynQcJQCcIi...","[something will give, Ever Seen, Birdwatcher, ...",3oldJCV4dggIdDCB8aFlFc,https://open.spotify.com/playlist/3oldJCV4dggI...,https://mosaic.scdn.co/640/ab67616d00001e0254c...,"[(4ztA5OPlOzUvbE0Cgh2gV9, something will give)..."
3,846 save me,"[3vlAqZvq7IYKOoOKwHGAxd, 0LXxEKF7GjUbGYCeBOPxb...","[double dog, Weirdwordsworms, Infinite Love wi...",3g39V9zZwlt9ZYn7JKTOCj,https://open.spotify.com/playlist/3g39V9zZwlt9...,https://image-cdn-ak.spotifycdn.com/image/ab67...,"[(3vlAqZvq7IYKOoOKwHGAxd, double dog), (0LXxEK..."
4,845 what did you expect,"[0rxQTgJzd0HCUk4QUxSkjK, 74OfKhcmck9KUHBPmCaOt...","[Even If It Hurts (feat. Blood Orange), Drifti...",0KgwFInw8xWYmtDQWayf2p,https://open.spotify.com/playlist/0KgwFInw8xWY...,https://image-cdn-ak.spotifycdn.com/image/ab67...,"[(0rxQTgJzd0HCUk4QUxSkjK, Even If It Hurts (fe..."


In [15]:
songs = songs.explode('zipp')
songs.head()

,name,track_ids,track_names,id,url,image,zipp
0,849 find out,"[6vca0aMELa1kc6mIs6w39c, 5jOCZRdWItMJKacbZolEJ...","[toothache, Death Grip, The fish song, Trailbl...",5oKwX8pOYWofuTNMKx4okR,https://open.spotify.com/playlist/5oKwX8pOYWof...,https://mosaic.scdn.co/640/ab67616d00001e02bd8...,"(6vca0aMELa1kc6mIs6w39c, toothache)"
0,849 find out,"[6vca0aMELa1kc6mIs6w39c, 5jOCZRdWItMJKacbZolEJ...","[toothache, Death Grip, The fish song, Trailbl...",5oKwX8pOYWofuTNMKx4okR,https://open.spotify.com/playlist/5oKwX8pOYWof...,https://mosaic.scdn.co/640/ab67616d00001e02bd8...,"(5jOCZRdWItMJKacbZolEJF, Death Grip)"
0,849 find out,"[6vca0aMELa1kc6mIs6w39c, 5jOCZRdWItMJKacbZolEJ...","[toothache, Death Grip, The fish song, Trailbl...",5oKwX8pOYWofuTNMKx4okR,https://open.spotify.com/playlist/5oKwX8pOYWof...,https://mosaic.scdn.co/640/ab67616d00001e02bd8...,"(6LJUlREHwzx1Z2BcqOdJSW, The fish song)"
0,849 find out,"[6vca0aMELa1kc6mIs6w39c, 5jOCZRdWItMJKacbZolEJ...","[toothache, Death Grip, The fish song, Trailbl...",5oKwX8pOYWofuTNMKx4okR,https://open.spotify.com/playlist/5oKwX8pOYWof...,https://mosaic.scdn.co/640/ab67616d00001e02bd8...,"(6WUPSfoTnqDimPiVI89lQ7, Trailblaze ☆)"
1,848 innocence,"[6MzofobZt2dm0Kf1hTThFz, 5HwHdYpk3QiOUhsfWKYkF...","[Diet Pepsi, Passionfruit, tears in the club (...",5sMFgzDK1HTjG2bxIyi9jI,https://open.spotify.com/playlist/5sMFgzDK1HTj...,https://mosaic.scdn.co/640/ab67616d00001e024e5...,"(6MzofobZt2dm0Kf1hTThFz, Diet Pepsi)"


In [16]:
songs['track_name'] = songs['zipp'].map(lambda x: x[1])
songs['track_id'] = songs['zipp'].map(lambda x: x[0])
songs.head()

,name,track_ids,track_names,id,url,image,zipp,track_name,track_id
0,849 find out,"[6vca0aMELa1kc6mIs6w39c, 5jOCZRdWItMJKacbZolEJ...","[toothache, Death Grip, The fish song, Trailbl...",5oKwX8pOYWofuTNMKx4okR,https://open.spotify.com/playlist/5oKwX8pOYWof...,https://mosaic.scdn.co/640/ab67616d00001e02bd8...,"(6vca0aMELa1kc6mIs6w39c, toothache)",toothache,6vca0aMELa1kc6mIs6w39c
0,849 find out,"[6vca0aMELa1kc6mIs6w39c, 5jOCZRdWItMJKacbZolEJ...","[toothache, Death Grip, The fish song, Trailbl...",5oKwX8pOYWofuTNMKx4okR,https://open.spotify.com/playlist/5oKwX8pOYWof...,https://mosaic.scdn.co/640/ab67616d00001e02bd8...,"(5jOCZRdWItMJKacbZolEJF, Death Grip)",Death Grip,5jOCZRdWItMJKacbZolEJF
0,849 find out,"[6vca0aMELa1kc6mIs6w39c, 5jOCZRdWItMJKacbZolEJ...","[toothache, Death Grip, The fish song, Trailbl...",5oKwX8pOYWofuTNMKx4okR,https://open.spotify.com/playlist/5oKwX8pOYWof...,https://mosaic.scdn.co/640/ab67616d00001e02bd8...,"(6LJUlREHwzx1Z2BcqOdJSW, The fish song)",The fish song,6LJUlREHwzx1Z2BcqOdJSW
0,849 find out,"[6vca0aMELa1kc6mIs6w39c, 5jOCZRdWItMJKacbZolEJ...","[toothache, Death Grip, The fish song, Trailbl...",5oKwX8pOYWofuTNMKx4okR,https://open.spotify.com/playlist/5oKwX8pOYWof...,https://mosaic.scdn.co/640/ab67616d00001e02bd8...,"(6WUPSfoTnqDimPiVI89lQ7, Trailblaze ☆)",Trailblaze ☆,6WUPSfoTnqDimPiVI89lQ7
1,848 innocence,"[6MzofobZt2dm0Kf1hTThFz, 5HwHdYpk3QiOUhsfWKYkF...","[Diet Pepsi, Passionfruit, tears in the club (...",5sMFgzDK1HTjG2bxIyi9jI,https://open.spotify.com/playlist/5sMFgzDK1HTj...,https://mosaic.scdn.co/640/ab67616d00001e024e5...,"(6MzofobZt2dm0Kf1hTThFz, Diet Pepsi)",Diet Pepsi,6MzofobZt2dm0Kf1hTThFz


In [17]:
songs.reset_index()

,index,name,track_ids,track_names,id,url,image,zipp,track_name,track_id
0,0,849 find out,"[6vca0aMELa1kc6mIs6w39c, 5jOCZRdWItMJKacbZolEJ...","[toothache, Death Grip, The fish song, Trailbl...",5oKwX8pOYWofuTNMKx4okR,https://open.spotify.com/playlist/5oKwX8pOYWof...,https://mosaic.scdn.co/640/ab67616d00001e02bd8...,"(6vca0aMELa1kc6mIs6w39c, toothache)",toothache,6vca0aMELa1kc6mIs6w39c
1,0,849 find out,"[6vca0aMELa1kc6mIs6w39c, 5jOCZRdWItMJKacbZolEJ...","[toothache, Death Grip, The fish song, Trailbl...",5oKwX8pOYWofuTNMKx4okR,https://open.spotify.com/playlist/5oKwX8pOYWof...,https://mosaic.scdn.co/640/ab67616d00001e02bd8...,"(5jOCZRdWItMJKacbZolEJF, Death Grip)",Death Grip,5jOCZRdWItMJKacbZolEJF
2,0,849 find out,"[6vca0aMELa1kc6mIs6w39c, 5jOCZRdWItMJKacbZolEJ...","[toothache, Death Grip, The fish song, Trailbl...",5oKwX8pOYWofuTNMKx4okR,https://open.spotify.com/playlist/5oKwX8pOYWof...,https://mosaic.scdn.co/640/ab67616d00001e02bd8...,"(6LJUlREHwzx1Z2BcqOdJSW, The fish song)",The fish song,6LJUlREHwzx1Z2BcqOdJSW
3,0,849 find out,"[6vca0aMELa1kc6mIs6w39c, 5jOCZRdWItMJKacbZolEJ...","[toothache, Death Grip, The fish song, Trailbl...",5oKwX8pOYWofuTNMKx4okR,https://open.spotify.com/playlist/5oKwX8pOYWof...,https://mosaic.scdn.co/640/ab67616d00001e02bd8...,"(6WUPSfoTnqDimPiVI89lQ7, Trailblaze ☆)",Trailblaze ☆,6WUPSfoTnqDimPiVI89lQ7
4,1,848 innocence,"[6MzofobZt2dm0Kf1hTThFz, 5HwHdYpk3QiOUhsfWKYkF...","[Diet Pepsi, Passionfruit, tears in the club (...",5sMFgzDK1HTjG2bxIyi9jI,https://open.spotify.com/playlist/5sMFgzDK1HTj...,https://mosaic.scdn.co/640/ab67616d00001e024e5...,"(6MzofobZt2dm0Kf1hTThFz, Diet Pepsi)",Diet Pepsi,6MzofobZt2dm0Kf1hTThFz
...,...,...,...,...,...,...,...,...,...,...
9415,926,"2022 releases i missed, whoopies!","[4euglpuBWHIqrTofPsdVTi, 337bOFFRgT6HyF2V4fLCY...","[Rotten, Mulholland Dr., Concrete Over Water, ...",1w1Jkxyxbd83lCaNeD1wkz,https://open.spotify.com/playlist/1w1Jkxyxbd83...,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(1IHWl5LamUGEuP4ozKQSXZ, Tití Me Preguntó)",Tití Me Preguntó,1IHWl5LamUGEuP4ozKQSXZ
9416,926,"2022 releases i missed, whoopies!","[4euglpuBWHIqrTofPsdVTi, 337bOFFRgT6HyF2V4fLCY...","[Rotten, Mulholland Dr., Concrete Over Water, ...",1w1Jkxyxbd83lCaNeD1wkz,https://open.spotify.com/playlist/1w1Jkxyxbd83...,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(3k2kIiJqUiIP49iUcOLpWT, The Place Where He In...",The Place Where He Inserted the Blade,3k2kIiJqUiIP49iUcOLpWT
9417,926,"2022 releases i missed, whoopies!","[4euglpuBWHIqrTofPsdVTi, 337bOFFRgT6HyF2V4fLCY...","[Rotten, Mulholland Dr., Concrete Over Water, ...",1w1Jkxyxbd83lCaNeD1wkz,https://open.spotify.com/playlist/1w1Jkxyxbd83...,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(2kv9BIaOOHZrIphec1U68O, Soft Engine)",Soft Engine,2kv9BIaOOHZrIphec1U68O
9418,926,"2022 releases i missed, whoopies!","[4euglpuBWHIqrTofPsdVTi, 337bOFFRgT6HyF2V4fLCY...","[Rotten, Mulholland Dr., Concrete Over Water, ...",1w1Jkxyxbd83lCaNeD1wkz,https://open.spotify.com/playlist/1w1Jkxyxbd83...,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(0I5OHUf0S6WoahmWRYHwEw, The Tisburys (On Main...",The Tisburys (On Main Street),0I5OHUf0S6WoahmWRYHwEw


In [18]:
songs = songs.dropna()

### Audio Features
- manipulate song df to get audio features from spotify api
- merge with song df for searchability
- take mean audio features as score for playlist overall
- merge with playlist df
- export dfs as csvs to avoid repeatedly calling api for analysis

In [19]:
def batch(lst, n):
    for i in range(0, len(lst), n):
        if len(lst) >= (i+n):
            yield lst[i:i + n]
        else:
            yield lst[i:]

def get_audio_features(uris):
    results = []
    pbar = tqdm(total=1000)
    batched = batch(uris, 100)
    for each in batched:
        results.append(sp.audio_features(each))
        pbar.update(50)
    pbar.close()
    return results

In [20]:
audio_features_results = get_audio_features(songs['track_id'])

4750it [00:21, 216.05it/s]                          


In [21]:
unpacked = [song for hundred in audio_features_results for song in hundred]

In [22]:
features = pd.DataFrame(unpacked)
features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.367,0.710,10,-7.899,0,0.0796,0.538,0.00849,0.1080,0.444,114.697,audio_features,6vca0aMELa1kc6mIs6w39c,spotify:track:6vca0aMELa1kc6mIs6w39c,https://api.spotify.com/v1/tracks/6vca0aMELa1k...,https://api.spotify.com/v1/audio-analysis/6vca...,208271,5
1,0.485,0.705,9,-7.528,1,0.0515,0.184,0.00000,0.2890,0.687,161.302,audio_features,5jOCZRdWItMJKacbZolEJF,spotify:track:5jOCZRdWItMJKacbZolEJF,https://api.spotify.com/v1/tracks/5jOCZRdWItMJ...,https://api.spotify.com/v1/audio-analysis/5jOC...,188840,4
2,0.481,0.265,4,-12.910,1,0.0329,0.887,0.00865,0.1020,0.221,86.775,audio_features,6LJUlREHwzx1Z2BcqOdJSW,spotify:track:6LJUlREHwzx1Z2BcqOdJSW,https://api.spotify.com/v1/tracks/6LJUlREHwzx1...,https://api.spotify.com/v1/audio-analysis/6LJU...,162759,4
3,0.916,0.854,9,-4.331,1,0.0344,0.435,0.21000,0.0878,0.925,122.011,audio_features,6WUPSfoTnqDimPiVI89lQ7,spotify:track:6WUPSfoTnqDimPiVI89lQ7,https://api.spotify.com/v1/tracks/6WUPSfoTnqDi...,https://api.spotify.com/v1/audio-analysis/6WUP...,192361,4
4,0.669,0.561,10,-6.538,1,0.0411,0.495,0.00962,0.0954,0.841,150.069,audio_features,6MzofobZt2dm0Kf1hTThFz,spotify:track:6MzofobZt2dm0Kf1hTThFz,https://api.spotify.com/v1/tracks/6MzofobZt2dm...,https://api.spotify.com/v1/audio-analysis/6Mzo...,169698,4


In [23]:
songs_features = songs.merge(features,  left_on='track_id', right_on='id')
songs_features.head()

,name,track_ids,track_names,id_x,url,image,zipp,track_name,track_id,danceability,...,liveness,valence,tempo,type,id_y,uri,track_href,analysis_url,duration_ms,time_signature
0,849 find out,"[6vca0aMELa1kc6mIs6w39c, 5jOCZRdWItMJKacbZolEJ...","[toothache, Death Grip, The fish song, Trailbl...",5oKwX8pOYWofuTNMKx4okR,https://open.spotify.com/playlist/5oKwX8pOYWof...,https://mosaic.scdn.co/640/ab67616d00001e02bd8...,"(6vca0aMELa1kc6mIs6w39c, toothache)",toothache,6vca0aMELa1kc6mIs6w39c,0.367,...,0.1080,0.444,114.697,audio_features,6vca0aMELa1kc6mIs6w39c,spotify:track:6vca0aMELa1kc6mIs6w39c,https://api.spotify.com/v1/tracks/6vca0aMELa1k...,https://api.spotify.com/v1/audio-analysis/6vca...,208271,5
1,849 find out,"[6vca0aMELa1kc6mIs6w39c, 5jOCZRdWItMJKacbZolEJ...","[toothache, Death Grip, The fish song, Trailbl...",5oKwX8pOYWofuTNMKx4okR,https://open.spotify.com/playlist/5oKwX8pOYWof...,https://mosaic.scdn.co/640/ab67616d00001e02bd8...,"(5jOCZRdWItMJKacbZolEJF, Death Grip)",Death Grip,5jOCZRdWItMJKacbZolEJF,0.485,...,0.2890,0.687,161.302,audio_features,5jOCZRdWItMJKacbZolEJF,spotify:track:5jOCZRdWItMJKacbZolEJF,https://api.spotify.com/v1/tracks/5jOCZRdWItMJ...,https://api.spotify.com/v1/audio-analysis/5jOC...,188840,4
2,849 find out,"[6vca0aMELa1kc6mIs6w39c, 5jOCZRdWItMJKacbZolEJ...","[toothache, Death Grip, The fish song, Trailbl...",5oKwX8pOYWofuTNMKx4okR,https://open.spotify.com/playlist/5oKwX8pOYWof...,https://mosaic.scdn.co/640/ab67616d00001e02bd8...,"(6LJUlREHwzx1Z2BcqOdJSW, The fish song)",The fish song,6LJUlREHwzx1Z2BcqOdJSW,0.481,...,0.1020,0.221,86.775,audio_features,6LJUlREHwzx1Z2BcqOdJSW,spotify:track:6LJUlREHwzx1Z2BcqOdJSW,https://api.spotify.com/v1/tracks/6LJUlREHwzx1...,https://api.spotify.com/v1/audio-analysis/6LJU...,162759,4
3,849 find out,"[6vca0aMELa1kc6mIs6w39c, 5jOCZRdWItMJKacbZolEJ...","[toothache, Death Grip, The fish song, Trailbl...",5oKwX8pOYWofuTNMKx4okR,https://open.spotify.com/playlist/5oKwX8pOYWof...,https://mosaic.scdn.co/640/ab67616d00001e02bd8...,"(6WUPSfoTnqDimPiVI89lQ7, Trailblaze ☆)",Trailblaze ☆,6WUPSfoTnqDimPiVI89lQ7,0.916,...,0.0878,0.925,122.011,audio_features,6WUPSfoTnqDimPiVI89lQ7,spotify:track:6WUPSfoTnqDimPiVI89lQ7,https://api.spotify.com/v1/tracks/6WUPSfoTnqDi...,https://api.spotify.com/v1/audio-analysis/6WUP...,192361,4
4,848 innocence,"[6MzofobZt2dm0Kf1hTThFz, 5HwHdYpk3QiOUhsfWKYkF...","[Diet Pepsi, Passionfruit, tears in the club (...",5sMFgzDK1HTjG2bxIyi9jI,https://open.spotify.com/playlist/5sMFgzDK1HTj...,https://mosaic.scdn.co/640/ab67616d00001e024e5...,"(6MzofobZt2dm0Kf1hTThFz, Diet Pepsi)",Diet Pepsi,6MzofobZt2dm0Kf1hTThFz,0.669,...,0.0954,0.841,150.069,audio_features,6MzofobZt2dm0Kf1hTThFz,spotify:track:6MzofobZt2dm0Kf1hTThFz,https://api.spotify.com/v1/tracks/6MzofobZt2dm...,https://api.spotify.com/v1/audio-analysis/6Mzo...,169698,4


In [24]:
playlist_features = songs_features[['name', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']].groupby('name', sort=False).median()
playlist_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
name,,,,,,,,,,,
849 find out,0.4830,0.7075,9.0,-7.7135,1.0,0.04295,0.48650,0.008570,0.1050,0.5655,118.3540
848 innocence,0.6425,0.6660,7.5,-6.0575,0.5,0.04825,0.12395,0.000063,0.1145,0.6500,138.6175
847 sick joke,0.5200,0.3220,6.0,-10.7800,1.0,0.03780,0.77500,0.000027,0.1260,0.3770,142.9120
846 save me,0.6520,0.6470,8.5,-7.4910,1.0,0.05930,0.20260,0.026150,0.1145,0.5010,129.0055
845 what did you expect,0.6430,0.5325,7.0,-9.1490,0.0,0.04580,0.29700,0.016465,0.1245,0.4820,117.8560


### Genre

In [25]:
def get_genre(playlist_artists):
    results = []
    if len(playlist_artists) <= 50:
        artists = sp.artists(playlist_artists)
        for artist in artists['artists']:
            genres = artist['genres']
            results.append(genres)
    else:
        playlist_artists_batched = batch(playlist_artists, len(playlist_artists)//50 + 1)
        for artist_batch in playlist_artists_batched:
            artists = sp.artists(artist_batch)
            for artist in artists['artists']:
                genres = artist['genres']
                results.append(genres)
    time.sleep(1)
    return [genre for genres in results for genre in genres]

In [26]:
filtered_artists = pd.Series([list(filter(lambda artist: artist is not None, p_artists)) for p_artists in artists])
filtered_artists

0      [0nugNBwdWaptgIAsEtx1It, 4Ky0ZDJDfY7UBbTVYPyVK...
1      [4gvjmrtzydbMpyJaXUtwvP, 2RqrWplViWHSGLzlhmDcb...
2      [3WWvqxymB3Nypxej8XcQKC, 35l9BRT7MXmM8bv2WDQiy...
3      [7HKjjFcvOZyupxbOMdmSwE, 4TZieE5978SbTInJswaay...
4      [1xcMOgFUM1IYZE22YjCvsL, 5FxD8fkQZ6KcsSYupDVoS...
                             ...                        
922    [6xdRb2GypJ7DqnWAI2mHGn, 1r1uxoy19fzMxunt3ONAk...
923    [3WrFJ7ztbogyGnTHbHJFl2, 5B7NeaqVrmXPyF05C9tnZ...
924    [5QdyldG4Fl4TPiOIeMNpBZ, 3h9TfIgwhovQELlP2jj4x...
925    [2gqMBdyddvN82dzZt4ZF14, 2rDxtYUzTAYJJE3Bl3Z5I...
926    [4vAQ4M7vgItwBtmBTgRu48, 6Gl4Q3ePw6HKMfIOix5Qp...
Length: 927, dtype: object

In [27]:
genres = playlist_features.copy().reset_index()
genres.head()

,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,849 find out,0.4830,0.7075,9.0,-7.7135,1.0,0.04295,0.48650,0.008570,0.1050,0.5655,118.3540
1,848 innocence,0.6425,0.6660,7.5,-6.0575,0.5,0.04825,0.12395,0.000063,0.1145,0.6500,138.6175
2,847 sick joke,0.5200,0.3220,6.0,-10.7800,1.0,0.03780,0.77500,0.000027,0.1260,0.3770,142.9120
3,846 save me,0.6520,0.6470,8.5,-7.4910,1.0,0.05930,0.20260,0.026150,0.1145,0.5010,129.0055
4,845 what did you expect,0.6430,0.5325,7.0,-9.1490,0.0,0.04580,0.29700,0.016465,0.1245,0.4820,117.8560


In [28]:
genres = pd.concat([genres, filtered_artists.rename('track_artists')], axis=1)
genres

,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_artists
0,849 find out,0.4830,0.7075,9.0,-7.7135,1.0,0.04295,0.48650,0.008570,0.1050,0.5655,118.3540,"[0nugNBwdWaptgIAsEtx1It, 4Ky0ZDJDfY7UBbTVYPyVK..."
1,848 innocence,0.6425,0.6660,7.5,-6.0575,0.5,0.04825,0.12395,0.000063,0.1145,0.6500,138.6175,"[4gvjmrtzydbMpyJaXUtwvP, 2RqrWplViWHSGLzlhmDcb..."
2,847 sick joke,0.5200,0.3220,6.0,-10.7800,1.0,0.03780,0.77500,0.000027,0.1260,0.3770,142.9120,"[3WWvqxymB3Nypxej8XcQKC, 35l9BRT7MXmM8bv2WDQiy..."
3,846 save me,0.6520,0.6470,8.5,-7.4910,1.0,0.05930,0.20260,0.026150,0.1145,0.5010,129.0055,"[7HKjjFcvOZyupxbOMdmSwE, 4TZieE5978SbTInJswaay..."
4,845 what did you expect,0.6430,0.5325,7.0,-9.1490,0.0,0.04580,0.29700,0.016465,0.1245,0.4820,117.8560,"[1xcMOgFUM1IYZE22YjCvsL, 5FxD8fkQZ6KcsSYupDVoS..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
922,00 think about it,0.5310,0.3570,2.0,-8.4800,1.0,0.03150,0.33900,0.000303,0.1120,0.3030,129.7530,"[6xdRb2GypJ7DqnWAI2mHGn, 1r1uxoy19fzMxunt3ONAk..."
923,00 remember,0.5140,0.4350,8.0,-8.2780,1.0,0.02940,0.44900,0.000302,0.1240,0.4410,122.6760,"[3WrFJ7ztbogyGnTHbHJFl2, 5B7NeaqVrmXPyF05C9tnZ..."
924,00 remind me,0.4350,0.2450,3.0,-11.3150,1.0,0.03380,0.59000,0.002260,0.1180,0.3610,80.5910,"[5QdyldG4Fl4TPiOIeMNpBZ, 3h9TfIgwhovQELlP2jj4x..."
925,00 forgetting,0.4350,0.3810,5.0,-9.4360,1.0,0.03300,0.71700,0.000006,0.1120,0.4610,118.3270,"[2gqMBdyddvN82dzZt4ZF14, 2rDxtYUzTAYJJE3Bl3Z5I..."


In [29]:
genres_batched = np.array_split(genres, len(genres)//4 + 1)

genres_list = []

for batch in genres_batched:
    genres_list.append(batch['track_artists'].apply(get_genre))

genres_list

/Users/amayalim/Projects/turntable/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


[0    [indie pop, modern indie folk, hyperpop, hypna...
 1    [social media pop, alternative r&b, art pop, e...
 2    [bedroom pop, bubblegrunge, indie pop, pov: in...
 3    [experimental indie, art pop, canadian electro...
 Name: track_artists, dtype: object,
 4    [alternative r&b, art pop, escape room, gauze ...
 5    [alt z, boston folk, indie pop, singer-songwri...
 6    [modern dream pop, art pop, escape room, indie...
 7    [latinx alternative, portland indie, bedroom p...
 Name: track_artists, dtype: object,
 8     [conscious hip hop, hip hop, rap, underground ...
 9     [indie pop, art pop, candy pop, metropopolis, ...
 10    [art pop, candy pop, metropopolis, pop, uk pop...
 11    [bedroom pop, indie pop, pov: indie, alternati...
 Name: track_artists, dtype: object,
 12    [art pop, experimental pop, uk alternative pop...
 13    [art pop, pop, australian hip hop, canadian co...
 14    [pop, classic country pop, country, country da...
 15    [5th wave emo, bubblegrunge, diy em

In [30]:
flattened = [playlist_genres for batch in genres_list for playlist_genres in batch]

In [31]:
genres['genres'] = flattened

In [32]:
genres

,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_artists,genres
0,849 find out,0.4830,0.7075,9.0,-7.7135,1.0,0.04295,0.48650,0.008570,0.1050,0.5655,118.3540,"[0nugNBwdWaptgIAsEtx1It, 4Ky0ZDJDfY7UBbTVYPyVK...","[indie pop, modern indie folk, hyperpop, hypna..."
1,848 innocence,0.6425,0.6660,7.5,-6.0575,0.5,0.04825,0.12395,0.000063,0.1145,0.6500,138.6175,"[4gvjmrtzydbMpyJaXUtwvP, 2RqrWplViWHSGLzlhmDcb...","[social media pop, alternative r&b, art pop, e..."
2,847 sick joke,0.5200,0.3220,6.0,-10.7800,1.0,0.03780,0.77500,0.000027,0.1260,0.3770,142.9120,"[3WWvqxymB3Nypxej8XcQKC, 35l9BRT7MXmM8bv2WDQiy...","[bedroom pop, bubblegrunge, indie pop, pov: in..."
3,846 save me,0.6520,0.6470,8.5,-7.4910,1.0,0.05930,0.20260,0.026150,0.1145,0.5010,129.0055,"[7HKjjFcvOZyupxbOMdmSwE, 4TZieE5978SbTInJswaay...","[experimental indie, art pop, canadian electro..."
4,845 what did you expect,0.6430,0.5325,7.0,-9.1490,0.0,0.04580,0.29700,0.016465,0.1245,0.4820,117.8560,"[1xcMOgFUM1IYZE22YjCvsL, 5FxD8fkQZ6KcsSYupDVoS...","[alternative r&b, art pop, escape room, gauze ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
922,00 think about it,0.5310,0.3570,2.0,-8.4800,1.0,0.03150,0.33900,0.000303,0.1120,0.3030,129.7530,"[6xdRb2GypJ7DqnWAI2mHGn, 1r1uxoy19fzMxunt3ONAk...","[alt z, indie pop, indie poptimism, la pop, me..."
923,00 remember,0.5140,0.4350,8.0,-8.2780,1.0,0.02940,0.44900,0.000302,0.1240,0.4410,122.6760,"[3WrFJ7ztbogyGnTHbHJFl2, 5B7NeaqVrmXPyF05C9tnZ...","[british invasion, classic rock, merseybeat, p..."
924,00 remind me,0.4350,0.2450,3.0,-11.3150,1.0,0.03380,0.59000,0.002260,0.1180,0.3610,80.5910,"[5QdyldG4Fl4TPiOIeMNpBZ, 3h9TfIgwhovQELlP2jj4x...","[art pop, brooklyn indie, chamber pop, country..."
925,00 forgetting,0.4350,0.3810,5.0,-9.4360,1.0,0.03300,0.71700,0.000006,0.1120,0.4610,118.3270,"[2gqMBdyddvN82dzZt4ZF14, 2rDxtYUzTAYJJE3Bl3Z5I...","[black americana, folk, new americana, uk amer..."


In [37]:
playlists.to_csv('csv_files/playlists.csv', index=False)
songs.to_csv('csv_files/songs.csv', index=False)
features.to_csv('csv_files/features.csv', index=False)

In [38]:
playlist_features.to_csv('csv_files/playlist_features.csv', index=True)

In [40]:
genres.to_csv('csv_files/genres.csv', index=False)